In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pymysql
import plotly.express as px

In [2]:
db=pymysql.connect(
    host="localhost",
    user="root",
    password="123456",
    database="ecommerce"
)
cur=db.cursor()

# Queries

##### List all unique cities where customers are located.

In [3]:
query="select distinct (customer_city) from customers"
cur.execute(query)
data=cur.fetchall()
data

(('franca',),
 ('sao bernardo do campo',),
 ('sao paulo',),
 ('mogi das cruzes',),
 ('campinas',),
 ('jaragua do sul',),
 ('timoteo',),
 ('curitiba',),
 ('belo horizonte',),
 ('montes claros',),
 ('rio de janeiro',),
 ('lencois paulista',),
 ('caxias do sul',),
 ('piracicaba',),
 ('guarulhos',),
 ('pacaja',),
 ('florianopolis',),
 ('aparecida de goiania',),
 ('santo andre',),
 ('goiania',),
 ('cachoeiro de itapemirim',),
 ('sao jose dos campos',),
 ('sao roque',),
 ('camacari',),
 ('resende',),
 ('sumare',),
 ('novo hamburgo',),
 ('sao luis',),
 ('sao jose',),
 ('santa barbara',),
 ('ribeirao preto',),
 ('ituiutaba',),
 ('taquarituba',),
 ('sao jose dos pinhais',),
 ('barrinha',),
 ('parati',),
 ('dourados',),
 ('trindade',),
 ('cascavel',),
 ('fortaleza',),
 ('brasilia',),
 ('pelotas',),
 ('porto alegre',),
 ('salto',),
 ('jundiai',),
 ('cacapava',),
 ('sao vicente',),
 ('uberlandia',),
 ('botelhos',),
 ('sao goncalo',),
 ('araucaria',),
 ('nova iguacu',),
 ('areia branca',),
 ('campo

#### Count the number of orders placed in 2017.

In [4]:
query="select count(order_id) from orders where order_purchase_timestamp between '2017-01-01' and '2017-12-31'"
cur.execute(query)
data=cur.fetchall()
print("total orders placed in 2017 are", data[0][0])

total orders placed in 2017 are 45027


####  Find the total sales per category.

In [5]:
query="""select products.product_category , round(sum(payments.payment_value),2) 
from products join order_items  
on products.product_id = order_items.product_id 
join payments on payments.order_id = order_items.order_id  
group by products.product_category"""
cur.execute(query)
data=cur.fetchall()
data

(('perfumery', 506738.66),
 ('Furniture Decoration', 1430176.39),
 ('telephony', 486882.05),
 ('Fashion Bags and Accessories', 218158.28),
 ('bed table bath', 1712553.67),
 ('automotive', 852294.33),
 ('computer accessories', 1585330.45),
 ('housewares', 1094758.13),
 ('babies', 539845.66),
 ('toys', 619037.69),
 ('Furniture office', 646826.49),
 ('Cool Stuff', 779698.0),
 ('HEALTH BEAUTY', 1657373.12),
 ('pet Shop', 311268.97),
 ('General Interest Books', 64694.76),
 ('stationary store', 317440.07),
 ('Watches present', 1429216.68),
 ('PCs', 279121.55),
 ('climatization', 91170.66),
 ('Construction Tools Construction', 241475.63),
 ('sport leisure', 1392127.56),
 ('Bags Accessories', 187151.29),
 ('electronics', 259857.1),
 ('Casa Construcao', 136645.29),
 ('Games consoles', 195480.38),
 ('home appliances', 95532.27),
 ('Garden tools', 838280.75),
 ('Agro Industria e Comercio', 118730.61),
 (None, 252801.71),
 ('audio', 60324.62),
 ('technical books', 24915.44),
 ('Construction Tools 

In [6]:
df=pd.DataFrame(data,columns=["product_category","total_sales"])
df=df.sort_values(by="total_sales",ascending=False)

In [7]:
px.bar(df,x="product_category",y="total_sales",title="Total sales of each product category")

#### Calculate the percentage of orders that were paid in installments

In [8]:
query="""select sum(case when payment_installments>1 then 1 else 0 end)*100/count(*) from payments"""
cur.execute(query)
data=cur.fetchall()
data

((Decimal('49.4176'),),)

#### Count the number of customers from each state

In [10]:
query="""select count(customer_unique_id) , customer_state from customers group by customer_state"""
cur.execute(query)
data=cur.fetchall()
df=pd.DataFrame(data,columns=["customer_count","customer_state"])
df=df.sort_values(by="customer_count",ascending=False)
df.head()

,customer_count,customer_state
0,41746,SP
4,12852,RJ
2,11635,MG
5,5466,RS
3,5045,PR


In [11]:
px.bar(df,x="customer_state",y="customer_count",title="Total customers in each state")


#### Calculate the number of orders per month in 2018.

In [12]:
query="""select count(order_id),monthname(order_purchase_timestamp) as months from orders where order_purchase_timestamp between '2018-01-01' and '2018-12-31' group by months """
cur.execute(query)
data=cur.fetchall()
df=pd.DataFrame(data,columns=["order_count","months"])
df=df.sort_values(by="order_count",ascending=False)
df

,order_count,months
5,7269,January
4,7211,March
7,6939,April
6,6873,May
2,6728,February
1,6512,August
0,6292,July
3,6167,June
8,16,September
9,4,October


In [13]:
px.bar(df,x="months",y="order_count",title="Total orders in each month of 2018")

####  Find the average number of products per order, grouped by customer city

In [14]:
query=""" with count_per_order as (select orders.order_id ,orders.customer_id , count(order_items.order_id) as OC
from orders join order_items on orders.order_id = order_items.order_id
group by orders.order_id,orders.customer_id) 

select customers.customer_city , round(AVG(count_per_order.OC),2) from
customers join count_per_order on customers.customer_id= count_per_order.customer_id group by customers.customer_city;



"""
cur.execute(query)
data=cur.fetchall()
df=pd.DataFrame(data,columns=["customer_city","average_products_per_order"])
df

,customer_city,average_products_per_order
0,sao jose dos campos,1.14
1,indaial,1.12
2,treze tilias,1.27
3,sao paulo,1.16
4,rio de janeiro,1.15
...,...,...
4105,buriti,3.00
4106,morro agudo de goias,1.00
4107,santa filomena,1.00
4108,guamiranga,1.00


#### Calculate the percentage of total revenue contributed by each product category

In [15]:
query="""select products.product_category ,sum(payments.payment_value) * 100/ (select sum(payment_value) from payments)
from products join order_items  
on products.product_id = order_items.product_id 
join payments on payments.order_id = order_items.order_id  
group by products.product_category"""
cur.execute(query)
data=cur.fetchall()
df=pd.DataFrame(data,columns=["product_category","percentage_sales"])
df

,product_category,percentage_sales
0,perfumery,3.165361
1,Furniture Decoration,8.933649
2,telephony,3.041326
3,Fashion Bags and Accessories,1.362734
4,bed table bath,10.697529
...,...,...
69,cds music dvds,0.007492
70,La Cuisine,0.018199
71,Fashion Children's Clothing,0.004908
72,PC Gamer,0.013583


####  Identify the correlation between product price and the number of times a product has been purchased.

In [17]:
query="""select p.product_category, count(o.product_id) , round(avg(o.price),2) 
from products p join order_items o 
on p.product_id=o.product_id
group by p.product_category;"""
cur.execute(query)
data=cur.fetchall()
df=pd.DataFrame(data,columns=["product_category","total_products_sold","average_price"])
df

,product_category,total_products_sold,average_price
0,HEALTH BEAUTY,9670,130.16
1,sport leisure,8641,114.34
2,Cool Stuff,3796,167.36
3,computer accessories,7827,116.51
4,Watches present,5991,201.14
...,...,...,...
69,flowers,33,33.64
70,Kitchen portable and food coach,15,264.57
71,House Comfort 2,30,25.34
72,CITTE AND UPHACK FURNITURE,38,114.95


In [18]:
px.scatter(df,x="total_products_sold",y="average_price",color="product_category",title="Total products sold vs average price")

In [21]:
arr1=df['total_products_sold']
arr2=df['average_price']
np.corrcoef(arr1,arr2)

array([[ 1.        , -0.10631514],
       [-0.10631514,  1.        ]])

####  Calculate the total revenue generated by each seller, and rank them by revenue.

In [30]:
query="""select * , dense_rank() over( order by revenue desc) as rn from  (select o.seller_id , sum(p.payment_value) as revenue
from order_items o join payments p on o.order_id=p.order_id
group by o.seller_id) as a"""
cur.execute(query)
data=cur.fetchall()
df=pd.DataFrame(data,columns=["seller_id","total_sales",'rank'])
df

,seller_id,total_sales,rank
0,7c67e1448b00f6e969d365cea6b010ab,507166.907302,1
1,1025f0e2d44d7041d6cf58b6550e0bfa,308222.039840,2
2,4a3ca9315b744ce9f8e9374361493884,301245.269765,3
3,1f50f920176fa81dab994f9023523100,290253.420128,4
4,53243585a1d6dc2643021fd1853d8905,284903.080498,5
...,...,...,...
3090,ad14615bdd492b01b0d97922e87cb87f,19.209999,3082
3091,702835e4b785b67a084280efca355756,18.559999,3083
3092,4965a7002cca77301c82d3f91b82e1a9,16.360001,3084
3093,77128dec4bec4878c37ab7d6169d6f26,15.220000,3085


####  Calculate the moving average of order values for each customer over their order history

In [32]:
query="""select customer_id, order_purchase_timestamp,payment_value,avg( payment_value) over(partition by customer_id order by order_purchase_timestamp rows between 2 preceding and current row ) as moving_avg
from
(select orders.customer_id, orders.order_purchase_timestamp , payments.payment_value 
from orders join payments on
orders.order_id = payments.order_id) as a;"""
cur.execute(query)
data=cur.fetchall()
df=pd.DataFrame(data,columns=["customer_id","order_purchase_timestamp","payment_value","moving_avg"])
df

,customer_id,order_purchase_timestamp,payment_value,moving_avg
0,00012a2ce6f8dcda20d059ce98491703,2017-11-14 16:08:26,114.74,114.739998
1,000161a058600d5901f007fab4c27140,2017-07-16 09:40:32,67.41,67.410004
2,0001fd6190edaaf884bcaf3d49edf079,2017-02-28 11:06:43,195.42,195.419998
3,0002414f95344307404f0ace7a26f1d5,2017-08-16 13:09:20,179.35,179.350006
4,000379cdec625522490c315e70c7a9fb,2018-04-02 13:42:17,107.01,107.010002
...,...,...,...,...
103881,fffecc9f79fd8c764f843e9951b11341,2018-03-29 16:59:26,71.23,27.120001
103882,fffeda5b6d849fbd39689bb92087f431,2018-05-22 13:36:02,63.13,63.130001
103883,ffff42319e9b2d713724ae527742af25,2018-06-13 16:57:05,214.13,214.130005
103884,ffffa3172527f765de70084a7e53aae8,2017-09-02 11:53:32,45.50,45.500000


#### Calculate the cumulative sales per month for each year

In [33]:
query="""# cumulative sales
select years, months ,payment,  sum(payment) over( order by years, months )  as cumulative_sales from
(select year(o.order_purchase_timestamp) as years ,month(o.order_purchase_timestamp) as months , round(sum(p.payment_value) ,2) as payment
from orders o join payments p on o.order_id=p.order_id
group by years , months ) as a
"""
cur.execute(query)
data=cur.fetchall()
df=pd.DataFrame(data,columns=["years","months","payment","cumulative_sales"])
df

,years,months,payment,cumulative_sales
0,2016,9,252.24,252.24
1,2016,10,59090.48,59342.72
2,2016,12,19.62,59362.34
3,2017,1,138488.04,197850.38
4,2017,2,291908.01,489758.39
5,2017,3,449863.60,939621.99
6,2017,4,417788.03,1357410.02
7,2017,5,592918.82,1950328.84
8,2017,6,511276.38,2461605.22
9,2017,7,592382.92,3053988.14


In [34]:
px.line(df,x="months",y="cumulative_sales",color="years",title="Cumulative sales over the years")

#### Calculate the year-over-year growth rate of total sales

In [35]:
query="""select  year(o.order_purchase_timestamp) as years ,sum(p.payment_value) 
from orders o join payments p 
on o.order_id =p.order_id
group by years
"""
cur.execute(query)
data=cur.fetchall()
df=pd.DataFrame(data,columns=["years","total_sales"])
df

,years,total_sales
0,2017,7.249747e+06
1,2018,8.699763e+06
2,2016,5.936234e+04


#### Calculate the retention rate of customers, defined as the percentage of customers who make another purchase within 6 months of their first purchase

In [37]:
query="""WITH a AS (
    SELECT customer_id, MIN(order_purchase_timestamp) AS first_order
    FROM orders
    GROUP BY customer_id
),
b AS (
    SELECT a.customer_id, COUNT(DISTINCT orders.order_purchase_timestamp) AS next_order
    FROM orders
    JOIN a ON orders.customer_id = a.customer_id 
    WHERE orders.order_purchase_timestamp > a.first_order 
      AND orders.order_purchase_timestamp < DATE_ADD(a.first_order, INTERVAL 24 MONTH)
    GROUP BY a.customer_id
)
SELECT 100 * COUNT(DISTINCT b.customer_id) / COUNT(DISTINCT a.customer_id) AS percentage
FROM a
LEFT JOIN b ON a.customer_id = b.customer_id

"""
cur.execute(query)
data=cur.fetchall()
df=pd.DataFrame(data,columns=["retention_rate"])
df

,retention_rate
0,0.0000


#### Identify the top 3 customers who spent the most money in each year.

In [38]:
query="""with a as (select o.customer_id,year(o.order_purchase_timestamp) as years,sum(p.payment_value) as payment, 
dense_rank() over(partition by year(o.order_purchase_timestamp) order by sum(p.payment_value) desc ) as d_rank
from orders o join payments p 
on o.order_id=p.order_id
group by o.customer_id,years)
select a.customer_id , a.payment, a.years , a.d_rank  from a where a.d_rank<=3

"""
cur.execute(query)
data=cur.fetchall()
df=pd.DataFrame(data,columns=["customer_id","payment","years","rank"])
df

,customer_id,payment,years,rank
0,a9dc96b027d1252bbac0a9b72d837fc6,1423.550049,2016,1
1,1d34ed25963d5aae4cf3d7f3a4cda173,1400.739990,2016,2
2,4a06381959b6670756de02e07b83815f,1227.780029,2016,3
3,1617b1357756262bfa56ab541c47bc16,13664.080078,2017,1
4,c6e2731c5b391845f6800c97401a43a9,6929.310059,2017,2
5,3fd6777bbce08a352fddd04e4a7cc8f6,6726.660156,2017,3
6,ec5b2ba62e574342386871631fafd3fc,7274.879883,2018,1
7,f48d464a0baaea338cb25f816991ab1f,6922.209961,2018,2
8,e0a2412720e9ea4f26c1ac985f6a7358,4809.439941,2018,3


In [41]:
px.bar(df,x="years",y="payment",color="rank",title="Top 3 customers in each year")